<a href="https://colab.research.google.com/github/Manaswi3104/Plant_Disease_Detection/blob/main/Copy_of_pdd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Requirements

In [1]:
%cd /content
!rm -rf yolov5


/content


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Clone the YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

# Install required dependencies
!pip install -U -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17488, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 17488 (delta 0), reused 0 (delta 0), pack-reused 17486 (from 2)
Receiving objects: 100% (17488/17488), 16.55 MiB | 27.97 MiB/s, done.
Resolving deltas: 100% (11995/11995), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.8 MB/s eta 0:00:

In [ ]:
!pip install -q roboflow ultralytics

# Imports

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sivm205/soybean-diseased-leaf-dataset")

print("Path to dataset files:", path)

100%|██████████| 1.93G/1.93G [00:51<00:00, 40.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/sivm205/soybean-diseased-leaf-dataset/versions/1


In [5]:
import os

# Replace 'path' with the actual variable name if needed
for root, dirs, files in os.walk(path):
    print("📁 Directory:", root)
    for name in files[:5]:  # print first 5 files per folder
        print("   📄", name)
    print("---")


📁 Directory: /root/.cache/kagglehub/datasets/sivm205/soybean-diseased-leaf-dataset/versions/1
---
📁 Directory: /root/.cache/kagglehub/datasets/sivm205/soybean-diseased-leaf-dataset/versions/1/septoria
   📄 Septoriah.jpg
   📄 DSC_0082.jpg
   📄 DSC_0080.jpg
   📄 DSC_0083.jpg
   📄 DSC_0081.jpg
---
📁 Directory: /root/.cache/kagglehub/datasets/sivm205/soybean-diseased-leaf-dataset/versions/1/Southern blight
   📄 DSC_0105.jpg
   📄 DSC_0043.jpg
   📄 DSC_0039.jpg
   📄 DSC_0044.jpg
   📄 DSC_0122.jpg
---
📁 Directory: /root/.cache/kagglehub/datasets/sivm205/soybean-diseased-leaf-dataset/versions/1/bacterial_blight
   📄 bb (18).jpg
   📄 bb (46).jpg
   📄 bb (30).jpg
   📄 bb (12).jpg
   📄 bb (4).jpg
---
📁 Directory: /root/.cache/kagglehub/datasets/sivm205/soybean-diseased-leaf-dataset/versions/1/powdery_mildew
   📄 aug_DSC_0078.jpg
   📄 aug_aug_aug_DSC_0059.jpg
   📄 DSC_0209.jpg
   📄 DSC_0210.jpg
   📄 DSC_0066.jpg
---
📁 Directory: /root/.cache/kagglehub/datasets/sivm205/soybean-diseased-leaf-dataset

In [8]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load dataset
data_path = '/root/.cache/kagglehub/datasets/sivm205/soybean-diseased-leaf-dataset/versions/1'
dataset = datasets.ImageFolder(root=data_path, transform=transform)

# Split into train and test sets, and load into DataLoader
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [9]:
!pip install split-folders


In [11]:
import splitfolders

splitfolders.ratio(
    input='/root/.cache/kagglehub/datasets/sivm205/soybean-diseased-leaf-dataset/versions/1',
    output='/content/yolov5/soybean_cls_split',
    seed=42,
    ratio=(.8, .2)  # 80% train, 20% val
)


Copying files: 701 files [00:13, 51.47 files/s]


In [12]:
%cd /content/yolov5

!python classify/train.py \
  --model yolov5s-cls.pt \
  --data soybean_cls_split \
  --epochs 20 \
  --img 224 \
  --batch 32


/content/yolov5
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
/content/yolov5/utils/general.py:32: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-06-11 06:00:59.494867: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been regi

In [16]:
!python classify/val.py \
  --weights runs/train-cls/exp/weights/best.pt \
  --data soybean_cls_split \
  --imgsz 224 \
  --batch-size 128 \
  --verbose



/content/yolov5/utils/general.py:32: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg
classify/val: data=soybean_cls_split, weights=['runs/train-cls/exp/weights/best.pt'], batch_size=128, imgsz=224, device=, workers=8, verbose=True, project=runs/val-cls, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-420-g0c99ce80 Python-3.11.13 torch-2.7.1+cu126 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 117 layers, 4179498 parameters, 0 gradients, 10.4 GFLOPs
validating:   0% 0/2 [00:00<?, ?it/s]/content/yolov5/classify/val.py:111: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=device.type != "cpu"):
validating: 100% 2/2 [00:14<00:00, 

In [18]:
!python classify/predict.py --weights runs/train-cls/exp/weights/best.pt --source /content/image2.jpeg


/content/yolov5/utils/general.py:32: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg
classify/predict: weights=['runs/train-cls/exp/weights/best.pt'], source=/content/image2.jpeg, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-420-g0c99ce80 Python-3.11.13 torch-2.7.1+cu126 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 117 layers, 4179498 parameters, 0 gradients, 10.4 GFLOPs
image 1/1 /content/image2.jpeg: 224x224 Sudden Death Syndrone 0.92, ferrugen 0.01, crestamento 0.01, septoria 0.01, brown_spot 0.01, 4.3ms
Speed: 0.4ms pre-process, 4.3ms inference, 18.

In [17]:
# If in Google Colab
from google.colab import files
files.download('runs/train-cls/exp/weights/best.pt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

YOLOv11 Fine-Tuning

In [ ]:
import os
print(os.path.exists('runs/train-cls/exp/weights/best.pt'))

True


In [ ]:
from pyngrok import ngrok
import threading
import time
import os

# Kill previous Streamlit processes (optional, prevents conflict)
os.system("pkill -f streamlit")

# Function to run Streamlit app
def run_app():
    os.system("streamlit run app.py")

# Start the Streamlit app in a separate thread
thread = threading.Thread(target=run_app)
thread.start()

# Give Streamlit time to start
time.sleep(5)

# Connect ngrok to the local Streamlit app
public_url = ngrok.connect("http://localhost:8501")
print("Streamlit app is running at:", public_url)


Streamlit app is running at: NgrokTunnel: "https://a58a-34-125-5-122.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!pkill -f ngrok


In [ ]:
!pkill -f ngrok
